# Deploying and Monitoring

In this notebook we will deploy the network traffic classification model that we have trained in the previous steps to Amazon SageMaker hosting, which will expose a fully-managed real-time endpoint to execute inferences.

Amazon SageMaker is adding new capabilities that monitor ML models while in production and detect deviations in data quality in comparison to a baseline dataset (e.g. training data set). They enable you to capture the metadata and the input and output for invocations of the models that you deploy with Amazon SageMaker. They also enable you to analyze the data and monitor its quality. 

We will deploy the model to a real-time endpoint with data capture enabled and start collecting some inference inputs/outputs. Then, we will create a baseline and finally enable model monitoring to compare inference data with respect to the baseline and analyze the quality.

First, we set some variables, including the AWS region we are working in, the IAM execution role of the notebook instance and the Amazon S3 bucket where we will store data and outputs.

In [1]:
import os
import boto3
import sagemaker

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
prefix = 'aim362'

print(region)
print(role)
print(bucket_name)

us-east-1
arn:aws:iam::806570384721:role/service-role/AmazonSageMaker-ExecutionRole-20191201T115647
sagemaker-us-east-1-806570384721


## Deployment with Data Capture

We are going to deploy the latest network traffic classification model that we have trained. To deploy a model using the SM Python SDK, we need to make sure we have the Amazon S3 URI where the model artifacts are stored and the URI of the Docker container that will be used for hosting this model.

First, let's determine the Amazon S3 URI of the model artifacts by using a couple of utility functions which query Amazon SageMaker service to get the latest training job whose name starts with 'nw-traffic-classification-xgb' and then describing the training job.

In [2]:
import boto3

def get_latest_training_job_name(base_job_name):
    client = boto3.client('sagemaker')
    response = client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', 
                                         SortOrder='Descending', StatusEquals='Completed')
    if len(response['TrainingJobSummaries']) > 0 :
        return response['TrainingJobSummaries'][0]['TrainingJobName']
    else:
        raise Exception('Training job not found.')

def get_training_job_s3_model_artifacts(job_name):
    client = boto3.client('sagemaker')
    response = client.describe_training_job(TrainingJobName=job_name)
    s3_model_artifacts = response['ModelArtifacts']['S3ModelArtifacts']
    return s3_model_artifacts

latest_training_job_name = get_latest_training_job_name('nw-traffic-classification-xgb')
print(latest_training_job_name)
model_path = get_training_job_s3_model_artifacts(latest_training_job_name)
print(model_path)

nw-traffic-classification-xgb-2020-02-04-19-56-23-672
s3://sagemaker-us-east-1-806570384721/aim362/output/nw-traffic-classification-xgb-2020-02-04-19-56-23-672/output/model.tar.gz


For this model, we are going to use the same XGBoost Docker container we used for training, which also offers inference capabilities. As a consequence, we can just create the XGBoostModel object of the Amazon SageMaker Python SDK and then invoke its .deploy() method to execute deployment.

We will also provide an entrypoint script to be invoked at deployment/inference time. The purpose of this code is deserializing and loading the XGB model. In addition, we are re-defining the output functions as we want to extract the class value from the default array output. For example, for class 3 the XGB container would output [3.] but we want to extract only the 3 value.

In [3]:
!pygmentize source_dir/deploy_xgboost.py

import os
import pickle as pkl
import numpy as np

from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)

def model_fn(model_dir):
    model_file = model_dir + '/model.bin'
    model = pkl.load(open(model_file, 'rb'))
    return model

def output_fn(prediction, accept):
    
    pred_array_value = np.array(prediction)
    pred_value = int(pred_array_value[0])
    
    return worker.Response(str(pred_value), accept, mimetype=accept)


Now we are ready to create the XGBoostModel object.

In [4]:
from time import gmtime, strftime
from sagemaker.xgboost import XGBoostModel

model_name = 'nw-traffic-classification-xgb-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

code_location = 's3://{0}/{1}/code'.format(bucket_name, prefix)
xgboost_model = XGBoostModel(model_data=model_path,
                             entry_point='deploy_xgboost.py',
                             source_dir='source_dir/',
                             name=model_name,
                             code_location=code_location,
                             framework_version='0.90-2',
                             role=role, 
                             sagemaker_session=sagemaker_session)

Finally we create an endpoint with data capture enabled, for monitoring the model data quality.
Data capture is enabled at enpoint configuration level for the Amazon SageMaker real-time endpoint. You can choose to capture the request payload, the response payload or both and captured data is stored in JSON format.

In [5]:
from time import gmtime, strftime
from sagemaker.model_monitor import DataCaptureConfig

s3_capture_upload_path = 's3://{}/{}/monitoring/datacapture'.format(bucket_name, prefix)
print(s3_capture_upload_path)

endpoint_name = 'nw-traffic-classification-xgb-ep-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)

pred = xgboost_model.deploy(initial_instance_count=1,
                            instance_type='ml.m5.xlarge',
                            endpoint_name=endpoint_name,
                            data_capture_config=DataCaptureConfig(
                                enable_capture=True,
                                sampling_percentage=100,
                                destination_s3_uri=s3_capture_upload_path))

s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture
nw-traffic-classification-xgb-ep-2020-02-04-20-24-20
-----------------!

After the deployment has been completed, we can leverage on the RealTimePredictor object to execute HTTPs requests against the deployed endpoint and get inference results.

In [10]:
from sagemaker.predictor import RealTimePredictor

pred = RealTimePredictor(endpoint_name)
pred.content_type = 'text/csv'
pred.accept = 'text/csv'

# Expecting class 4
test_values = "80,1056736,3,4,20,964,20,0,6.666666667,11.54700538,964,0,241.0,482.0,931.1691850999999,6.6241710320000005,176122.6667,\
431204.4454,1056315,2,394,197.0,275.77164469999997,392,2,1056733,352244.3333,609743.1115,1056315,24,0,0,0,0,72,92,\
2.8389304419999997,3.78524059,0,964,123.0,339.8873763,115523.4286,0,0,1,1,0,0,0,1,1.0,140.5714286,6.666666667,\
241.0,0.0,0.0,0.0,0.0,0.0,0.0,3,20,4,964,8192,211,1,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0"

result = pred.predict(test_values)
print(result)

# Expecting class 7
test_values = "80,10151,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,197.0249237,10151.0,0.0,10151,10151,10151,10151.0,0.0,10151,10151,0,0.0,\
0.0,0,0,0,0,0,0,40,0,197.0249237,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,32738,\
-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,21,2,2018,2,0,1,0"

result = pred.predict(test_values)
print(result)

# Expecting class 0
test_values = "80,54322832,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0368169318,54322832.0,0.0,54322832,54322832,54322832,54322832.0,0.0,\
54322832,54322832,0,0.0,0.0,0,0,0,0,0,0,40,0,0.0368169318,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,\
0.0,0.0,0.0,0.0,2,0,0,0,279,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,23,2,2018,4,0,1,0"

result = pred.predict(test_values)
print(result)

b'4'
b'7'
b'0'


Now let's list the data capture files stored in S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred.

**Note that the delivery of capture data to Amazon S3 can require a couple of minutes so next cell might error. If this happens, please retry after a minute.**

In [14]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/monitoring/datacapture/{}'.format(prefix, endpoint_name)

result = s3_client.list_objects(Bucket=bucket_name, Prefix=current_endpoint_capture_prefix)
capture_files = ['s3://{0}/{1}'.format(bucket_name, capture_file.get("Key")) for capture_file in result.get('Contents')]

print("Capture Files: ")
print("\n ".join(capture_files))

Capture Files: 
s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/33-57-898-0815a2e8-c235-4891-bd5d-91a59c41b6ca.jsonl


We can also read the contents of one of these files and see how capture records are organized in JSON lines format.

In [15]:
!aws s3 cp {capture_files[0]} datacapture/captured_data_example.jsonl
!head datacapture/captured_data_example.jsonl

download: s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/33-57-898-0815a2e8-c235-4891-bd5d-91a59c41b6ca.jsonl to datacapture/captured_data_example.jsonl
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"80,1056736,3,4,20,964,20,0,6.666666667,11.54700538,964,0,241.0,482.0,931.1691850999999,6.6241710320000005,176122.6667,431204.4454,1056315,2,394,197.0,275.77164469999997,392,2,1056733,352244.3333,609743.1115,1056315,24,0,0,0,0,72,92,2.8389304419999997,3.78524059,0,964,123.0,339.8873763,115523.4286,0,0,1,1,0,0,0,1,1.0,140.5714286,6.666666667,241.0,0.0,0.0,0.0,0.0,0.0,0.0,3,20,4,964,8192,211,1,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"4","encoding":"CSV"}},"eventMetadata":{"eventId":"1e4cf25e-310d-4d6e-b9fa-5aea6b9a1f82","inferenceTime":"2020-0

In addition, we can better understand the content of each JSON line like follows:

In [16]:
import json
with open ("datacapture/captured_data_example.jsonl", "r") as myfile:
    data=myfile.read()

print(json.dumps(json.loads(data.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "80,1056736,3,4,20,964,20,0,6.666666667,11.54700538,964,0,241.0,482.0,931.1691850999999,6.6241710320000005,176122.6667,431204.4454,1056315,2,394,197.0,275.77164469999997,392,2,1056733,352244.3333,609743.1115,1056315,24,0,0,0,0,72,92,2.8389304419999997,3.78524059,0,964,123.0,339.8873763,115523.4286,0,0,1,1,0,0,0,1,1.0,140.5714286,6.666666667,241.0,0.0,0.0,0.0,0.0,0.0,0.0,3,20,4,964,8192,211,1,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "4",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "1e4cf25e-310d-4d6e-b9fa-5aea6b9a1f82",
    "inferenceTime": "2020-02-04T20:33:57Z"
  },
  "eventVersion": "0"
}


For each inference request, we get input data, output data and some metadata like the inference time captured and saved.

## Baselining

From our validation dataset let's ask Amazon SageMaker to suggest a set of baseline constraints and generate descriptive statistics for our features. Note that we are using the validation dataset for this workshop to make sure baselining time is short, and that file extension needs to be changed since the baselining jobs require .CSV file extension as default.
In reality, you might be willing to use a larger dataset as baseline.

In [17]:
import boto3

s3 = boto3.resource('s3')

bucket_key_prefix = "aim362/data/val/"
bucket = s3.Bucket(bucket_name)

for s3_object in bucket.objects.filter(Prefix=bucket_key_prefix):
    target_key = s3_object.key.replace('data/val/', 'monitoring/baselining/data/').replace('.part', '.csv')
    print('Copying {0} to {1} ...'.format(s3_object.key, target_key))
    
    copy_source = {
        'Bucket': bucket_name,
        'Key': s3_object.key
    }
    s3.Bucket(bucket_name).copy(copy_source, target_key)

Copying aim362/data/val/0.part to aim362/monitoring/baselining/data/0.csv ...
Copying aim362/data/val/1.part to aim362/monitoring/baselining/data/1.csv ...
Copying aim362/data/val/2.part to aim362/monitoring/baselining/data/2.csv ...
Copying aim362/data/val/3.part to aim362/monitoring/baselining/data/3.csv ...
Copying aim362/data/val/4.part to aim362/monitoring/baselining/data/4.csv ...
Copying aim362/data/val/5.part to aim362/monitoring/baselining/data/5.csv ...
Copying aim362/data/val/6.part to aim362/monitoring/baselining/data/6.csv ...
Copying aim362/data/val/7.part to aim362/monitoring/baselining/data/7.csv ...
Copying aim362/data/val/8.part to aim362/monitoring/baselining/data/8.csv ...
Copying aim362/data/val/9.part to aim362/monitoring/baselining/data/9.csv ...


In [18]:
baseline_data_path = 's3://{0}/{1}/monitoring/baselining/data'.format(bucket_name, prefix)
baseline_results_path = 's3://{0}/{1}/monitoring/baselining/results'.format(bucket_name, prefix)

print(baseline_data_path)
print(baseline_results_path)

s3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/data
s3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results


Please note that running the baselining job will require 8-10 minutes. In the meantime, you can take a look at the Deequ library, used to execute these analyses with the default Model Monitor container: https://github.com/awslabs/deequ

In [19]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.c5.4xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

In [20]:
my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_path,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_path,
    wait=True
)


Job Name:  baseline-suggestion-job-2020-02-04-20-36-40-816
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/data', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
...................2020-02-04 20:39:44,924 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:806570384721:processing-job/baseline-suggestion-job-2020-02-04-20-36-40-816', 'ProcessingJobName': 'baseline-suggestion-job-2020-02-04-20-36-40-816', 'Environment': {'dataset_format': '{"csv": {"header": true, "output_columns_position": "START"}}', 'dataset_sour

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

Let's display the statistics that were generated by the baselining job.

In [21]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,inferred_type,name,numerical_statistics.common.num_missing,numerical_statistics.common.num_present,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.data,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.max,numerical_statistics.mean,numerical_statistics.min,numerical_statistics.std_dev,numerical_statistics.sum
0,Integral,Target,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 1.4, 'cou...","[[], [], [14.0], [], [14.0, 0.0, 0.0, 0.0, 0.0...",0.64,2048.0,14.0,4.722893e+00,0.0,5.044772e+00,2.505646e+06
1,Integral,Dst Port,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 6553.0, '...","[[], [], [65480.0], [], [64977.0, 0.0, 0.0, 21...",0.64,2048.0,65530.0,5.249104e+03,0.0,1.421226e+04,2.784818e+09
2,Integral,Flow Duration,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 11999999....","[[], [], [119995979.0], [], [119877467.0, 0.0,...",0.64,2048.0,119999998.0,7.752425e+06,0.0,2.396173e+07,4.112910e+12
3,Integral,Tot Fwd Pkts,0,530532,"[{'lower_bound': 1.0, 'upper_bound': 28005.2, ...","[[], [], [121026.0], [], [122.0, 1.0, 1.0, 1.0...",0.64,2048.0,280043.0,9.729206e+01,1.0,3.424388e+03,5.161655e+07
4,Integral,Tot Bwd Pkts,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 2240.1, '...","[[], [], [975.0], [], [354.0, 0.0, 0.0, 0.0, 0...",0.64,2048.0,22401.0,4.103854e+00,0.0,9.392279e+01,2.177226e+06
5,Integral,TotLen Fwd Pkts,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 896137.6,...","[[], [], [3872832.0], [], [19444.0, 0.0, 0.0, ...",0.64,2048.0,8961376.0,3.256939e+03,0.0,1.098343e+05,1.727910e+09
6,Integral,TotLen Bwd Pkts,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 3267151.1...","[[], [], [1396234.0], [], [442364.0, 0.0, 0.0,...",0.64,2048.0,32671511.0,2.222791e+03,0.0,1.352153e+05,1.179262e+09
7,Integral,Fwd Pkt Len Max,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 187.3, 'c...","[[], [], [1460.0], [], [1460.0, 0.0, 0.0, 0.0,...",0.64,2048.0,1873.0,1.389074e+02,0.0,2.504418e+02,7.369484e+07
8,Integral,Fwd Pkt Len Min,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 146.0, 'c...","[[], [], [500.0], [], [315.0, 0.0, 0.0, 0.0, 0...",0.64,2048.0,1460.0,5.794467e+00,0.0,1.879376e+01,3.074150e+06
9,Fractional,Fwd Pkt Len Mean,0,530532,"[{'lower_bound': 0.0, 'upper_bound': 146.0, 'c...","[[], [], [659.8], [], [341.12280699999997, 0.0...",0.64,2048.0,1460.0,3.496471e+01,0.0,5.319775e+01,1.854990e+07


Then, we can also visualize the constraints.

In [22]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,completeness,inferred_type,name,num_constraints.is_non_negative
0,1.0,Integral,Target,True
1,1.0,Integral,Dst Port,True
2,1.0,Integral,Flow Duration,True
3,1.0,Integral,Tot Fwd Pkts,True
4,1.0,Integral,Tot Bwd Pkts,True
5,1.0,Integral,TotLen Fwd Pkts,True
6,1.0,Integral,TotLen Bwd Pkts,True
7,1.0,Integral,Fwd Pkt Len Max,True
8,1.0,Integral,Fwd Pkt Len Min,True
9,1.0,Fractional,Fwd Pkt Len Mean,True


#### Results

The baselining job has inspected the validation dataset and generated constraints and statistics, that will be used to monitor our endpoint.

## Generating violations artificially

In order to get some result relevant to monitoring analysis, we are going to generate artificially some inferences with feature values causing specific violations, and then invoke the endpoint with this data.

This requires about 2 minutes for 1000 inferences.

In [23]:
import time
import numpy as np
dist_values = np.random.normal(1, 0.2, 1000)

# Tot Fwd Pkts -> set to float (expected integer) [second feature]
# Flow Duration -> set to empty (missing value) [third feature]
# Fwd Pkt Len Mean -> sampled from random normal distribution [nineth feature]

artificial_values = "22,,40.3,0,0,0,0,0,{0},0.0,0,0,0.0,0.0,0.0,0.0368169318,54322832.0,0.0,54322832,54322832,54322832,54322832.0,0.0,\
54322832,54322832,0,0.0,0.0,0,0,0,0,0,0,40,0,0.0368169318,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,\
0.0,0.0,0.0,0.0,2,0,0,0,279,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,23,2,2018,4,0,1,0"

for i in range(1000):
    pred.predict(artificial_values.format(str(dist_values[i])))
    time.sleep(0.15)
    if i > 0 and i % 100 == 0 :
        print('Executed {0} inferences.'.format(i))

Executed 100 inferences.
Executed 200 inferences.
Executed 300 inferences.
Executed 400 inferences.
Executed 500 inferences.
Executed 600 inferences.
Executed 700 inferences.
Executed 800 inferences.
Executed 900 inferences.


## Monitoring

Once we have built the baseline for our data, we can enable endpoint monitoring by creating a monitoring schedule.
When the schedule fires, a monitoring job will be kicked-off and will inspect the data captured at the endpoint with respect to the baseline; then it will generate some report files that can be used to analyze monitoring results.

### Create Monitoring Schedule

Let's create the monitoring schedule for the previously created endpoint. When we create the schedule, we can also specify two scripts that will preprocess the records before the analysis takes place and execute post-processing at the end.
For this example, we are not going to use a record preprocessor, and we are just specifying a post-processor that outputs some text for demo purposes.

In [24]:
!pygmentize postprocessor.py

def postprocess_handler():
    print("Hello from post-proc script!")


We copy the script to Amazon S3 and specify the path where the monitoring reports will be saved to.

In [26]:
import boto3

monitoring_code_prefix = '{0}/monitoring/code'.format(prefix)
print(monitoring_code_prefix)

boto3.Session().resource('s3').Bucket(bucket_name).Object(monitoring_code_prefix + '/postprocessor.py').upload_file('postprocessor.py')
postprocessor_path = 's3://{0}/{1}/monitoring/code/postprocessor.py'.format(bucket_name, prefix)
print(postprocessor_path)

reports_path = 's3://{0}/{1}/monitoring/reports'.format(bucket_name, prefix)
print(reports_path)

aim362/monitoring/code
s3://sagemaker-us-east-1-806570384721/aim362/monitoring/code/postprocessor.py
s3://sagemaker-us-east-1-806570384721/aim362/monitoring/reports


Finally, we create the monitoring schedule with hourly schedule execution.

In [27]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

endpoint_name = pred.endpoint

mon_schedule_name = 'nw-traffic-classification-xgb-mon-sch-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=endpoint_name,
    post_analytics_processor_script=postprocessor_path,
    output_s3_uri=reports_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True
)


Creating Monitoring Schedule with name: nw-traffic-classification-xgb-mon-sch-2020-02-04-20-59-51


### Describe Monitoring Schedule

In [28]:
desc_schedule_result = my_default_monitor.describe_schedule()
desc_schedule_result

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:806570384721:monitoring-schedule/nw-traffic-classification-xgb-mon-sch-2020-02-04-20-59-51',
 'MonitoringScheduleName': 'nw-traffic-classification-xgb-mon-sch-2020-02-04-20-59-51',
 'MonitoringScheduleStatus': 'Pending',
 'CreationTime': datetime.datetime(2020, 2, 4, 20, 59, 51, 967000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 2, 4, 20, 59, 51, 997000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinition': {'BaselineConfig': {'ConstraintsResource': {'S3Uri': 's3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results/constraints.json'},
    'StatisticsResource': {'S3Uri': 's3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results/statistics.json'}},
   'MonitoringInputs': [{'EndpointInput': {'EndpointName': 'nw-traffic-classification-xgb-ep-2020-02-04-20-24-20',
      'LocalPath': '/opt/ml/pr

### Delete Monitoring Schedule

Once the schedule is created, it will kick of jobs at specified intervals. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. 
You might have to wait till you cross the hour boundary (in UTC) to see executions kick off. Since we don't want to wait for the hour in this example we can delete the schedule and use the code in next steps to simulate what will happen when a schedule is triggered, by running an Amazon SageMaker Processing Job.

In [29]:
# Note: this is just for the purpose of running this example.
my_default_monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: nw-traffic-classification-xgb-mon-sch-2020-02-04-20-59-51


### Triggering execution manually

In oder to trigger the execution manually, we first get all paths to data capture, baseline statistics, baseline constraints, etc.
Then, we use a utility fuction, defined in <a href="./monitoringjob_utils.py">monitoringjob_utils.py</a>, to run the processing job.

In [30]:
result = s3_client.list_objects(Bucket=bucket_name, Prefix=current_endpoint_capture_prefix)
capture_files = ['s3://{0}/{1}'.format(bucket_name, capture_file.get("Key")) for capture_file in result.get('Contents')]

print("Capture Files: ")
print("\n ".join(capture_files))

data_capture_path = capture_files[len(capture_files) - 1][: capture_files[len(capture_files) - 1].rfind('/')]
statistics_path = baseline_results_path + '/statistics.json'
constraints_path = baseline_results_path + '/constraints.json'

print(data_capture_path)
print(postprocessor_path)
print(statistics_path)
print(constraints_path)
print(reports_path)

Capture Files: 
s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/33-57-898-0815a2e8-c235-4891-bd5d-91a59c41b6ca.jsonl
 s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/51-28-839-f722c428-f4a3-48a1-a6ab-a5f695d3c99a.jsonl
 s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/52-28-908-839decf0-a61a-48d0-9b34-f8d50576eea6.jsonl
 s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/53-28-991-e0c9ae91-1350-405e-9929-df8a0c8eda9b.jsonl
s3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20
s3://sagemaker-us-east-1-806570384721/a

In [31]:
from monitoringjob_utils import run_model_monitor_job_processor

run_model_monitor_job_processor(region, 'ml.m5.xlarge', role, data_capture_path, statistics_path, constraints_path, reports_path,
                                postprocessor_path=postprocessor_path)


Job Name:  sagemaker-model-monitor-analyzer-2020-02-04-21-00-30-633
Inputs:  [{'InputName': 'input_1', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-806570384721/aim362/monitoring/datacapture/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20', 'LocalPath': '/opt/ml/processing/input/endpoint/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'baseline', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results/statistics.json', 'LocalPath': '/opt/ml/processing/baseline/stats', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'constraints', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results/constraints.json', 'LocalPath':

### Analysis

When the monitoring job completes, monitoring reports are saved to Amazon S3. Let's list the generated reports.

In [32]:
s3_client = boto3.Session().client('s3')
monitoring_reports_prefix = '{}/monitoring/reports/{}'.format(prefix, pred.endpoint)

result = s3_client.list_objects(Bucket=bucket_name, Prefix=monitoring_reports_prefix)
try:
    monitoring_reports = ['s3://{0}/{1}'.format(bucket_name, capture_file.get("Key")) for capture_file in result.get('Contents')]
    print("Monitoring Reports Files: ")
    print("\n ".join(monitoring_reports))
except:
    print('No monitoring reports found.')

Monitoring Reports Files: 
s3://sagemaker-us-east-1-806570384721/aim362/monitoring/reports/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/constraint_violations.json
 s3://sagemaker-us-east-1-806570384721/aim362/monitoring/reports/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/constraints.json
 s3://sagemaker-us-east-1-806570384721/aim362/monitoring/reports/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/statistics.json


We then copy monitoring reports locally.

In [33]:
!aws s3 cp {monitoring_reports[0]} monitoring/
!aws s3 cp {monitoring_reports[1]} monitoring/
!aws s3 cp {monitoring_reports[2]} monitoring/

download: s3://sagemaker-us-east-1-806570384721/aim362/monitoring/reports/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/constraint_violations.json to monitoring/constraint_violations.json
download: s3://sagemaker-us-east-1-806570384721/aim362/monitoring/reports/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/constraints.json to monitoring/constraints.json
download: s3://sagemaker-us-east-1-806570384721/aim362/monitoring/reports/nw-traffic-classification-xgb-ep-2020-02-04-20-24-20/AllTraffic/2020/02/04/20/statistics.json to monitoring/statistics.json


Let's display the violations identified by the monitoring execution.

In [34]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

file = open('monitoring/constraint_violations.json', 'r')
data = file.read()

violations_df = pd.io.json.json_normalize(json.loads(data)['violations'])
violations_df.head(10)

,constraint_check_type,description,feature_name
0,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.8919722497522299% of data is Integral.",Tot Fwd Pkts
1,completeness_check,Data completeness requirement is not met. Expected: 100.0%. Observed: Only 0.8919722497522299%.,Flow Duration
2,baseline_drift_check,Baseline drift distance: 0.9293200129316993 exceeds threshold: 0.1,Fwd Pkt Len Mean


We can see that the violations identified correspond to the ones that we artificially generated and that there is a feature that is generating some drift from the baseline.

### Advanced Hints

You might be asking yourself what are the type of violations that are monitored and how drift from the baseline is computed.

The types of violations monitored are listed here: https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-interpreting-violations.html. Most of them use configurable thresholds, that are specified in the monitoring configuration section of the baseline constraints JSON. Let's take a look at this configuration from the baseline constraints file:

In [35]:
!aws s3 cp {statistics_path} baseline/
!aws s3 cp {constraints_path} baseline/

download: s3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results/statistics.json to baseline/statistics.json
download: s3://sagemaker-us-east-1-806570384721/aim362/monitoring/baselining/results/constraints.json to baseline/constraints.json


In [36]:
import json
with open ("baseline/constraints.json", "r") as myfile:
    data=myfile.read()

print(json.dumps(json.loads(data)['monitoring_config'], indent=2))

{
  "evaluate_constraints": "Enabled",
  "emit_metrics": "Enabled",
  "datatype_check_threshold": 1.0,
  "domain_content_threshold": 1.0,
  "distribution_constraints": {
    "perform_comparison": "Enabled",
    "comparison_threshold": 0.1,
    "comparison_method": "Robust"
  }
}


This configuration is intepreted when the monitoring job is executed and used to compare captured data to the baseline. If you want to customize this section, you will have to update the **constraints.json** file and upload it back to Amazon S3 before launching the monitoring job.

When data distributions are compared to detect potential drift, you can choose to use either a _Simple_ or _Robust_ comparison method, where the latter has to be preferred when dealing with small datasets. Additional info: https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-byoc-constraints.html.

## Delete Endpoint

Finally we can delete the endpoint to free-up resources.

In [37]:
pred.delete_endpoint()
pred.delete_model()

## References

A Realistic Cyber Defense Dataset (CSE-CIC-IDS2018) https://registry.opendata.aws/cse-cic-ids2018/